In [90]:
import re
import mysql.connector
import pymysql
import os
from dotenv import load_dotenv
from konlpy.tag import Okt
from soynlp.noun import NewsNounExtractor
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2

In [91]:
class searchIssue :
    
    def __init__(self):
        load_dotenv()
        self.host=os.getenv("HOST")
        self.user=os.getenv("USER_NAME")
        self.password=os.getenv("PASSWORD")
        self.database=os.getenv("DATABASE")
    
    def hotEnter(self):
        
        articles = []
        
        cnn = pymysql.connect(host=self.host, user=self.user, password=self.password, database=self.database)
        cur = cnn.cursor()

        searchMaxQuery = 'SELECT * FROM enterprise_data WHERE devi_yes_avg = (SELECT MAX(devi_yes_avg) FROM enterprise_data)'

        cur.execute(searchMaxQuery)
        
        company = list(cur.fetchall()[0])
        
        hotEId = company[0]
        #print(hotEId)
        
        findArticlesQuery = 'SELECT news_doc FROM news_data WHERE enter_id = %s'
        cur.execute(findArticlesQuery,hotEId)
        
        findArticles = cur.fetchall()
        
        for i in range(len(findArticles)):
            articles.append(findArticles[i][0])
        
        key = self.keyWord(articles)
        
        company.append(key)
        
        cur.close()
        cnn.close()
        
        return company[1:]
    
    
    
    def coldEnter(self):
        
        articles = []
        
        cnn = pymysql.connect(host=self.host, user=self.user, password=self.password, database=self.database)
        cur = cnn.cursor()

        searchMaxQuery = 'SELECT * FROM enterprise_data WHERE devi_yes_avg = (SELECT MIN(devi_yes_avg) FROM enterprise_data)'

        cur.execute(searchMaxQuery)
        
        company = list(cur.fetchall()[0])
        
        coldEId = company[0]
        #print(hotEId)
        
        findArticlesQuery = 'SELECT news_doc FROM news_data WHERE enter_id = %s'
        cur.execute(findArticlesQuery,coldEId)
        
        findArticles = cur.fetchall()
        
        for i in range(len(findArticles)):
            articles.append(findArticles[i][0])
        
        key = self.keyWord(articles)
        
        company.append(key)
        
        cur.close()
        cnn.close()
        
        return company[1:]
        
        
        
    def keyWord(self,art):
        keyword_article = ''.join(art)

        tokenizer = Okt()
        raw_pos_tagged = tokenizer.pos(keyword_article, norm=True, stem=True)

        del_list = ['하다', '있다', '되다', '이다', '돼다', '않다', '그렇다', '아니다', '이렇다', '그렇다', '어떻다']

        word_cleaned = []
        for word in raw_pos_tagged:
            if not word[1] in ["Josa", "Eomi", "Punctuation", "Foreign"]: # Foreign == ”, “ 와 같이 제외되어야할 항목들
                if (len(word[0]) != 1) and (word[0] not in del_list): # 한 글자로 이뤄진 단어들을 제외 & 원치 않는 단어들을 제외
            # 숫자나 이메일 형식의 단어 제외
                    if not re.match(r'^[0-9]*$', word[0]) and not re.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', word[0]):
                        word_cleaned.append(word[0])

        word_dic = {}

        for word in word_cleaned:
            if word not in word_dic:
                word_dic[word] = 1 # changed from "0" to "1"
            else:
                word_dic[word] += 1

        sorted_word_dic = sorted(word_dic.items(), key=lambda x:x[1], reverse=True)[:5]

        return sorted_word_dic
    
    
    def test(self):
        
        articles = []
        
        cnn = pymysql.connect(host=self.host, user=self.user, password=self.password, database=self.database)
        cur = cnn.cursor()

        searchMaxQuery = 'SELECT * FROM enterprise_data WHERE devi_yes_avg = (SELECT MIN(devi_yes_avg) FROM enterprise_data)'

        cur.execute(searchMaxQuery)
        
        company = list(cur.fetchall()[0])
        
        coldEId = company[0]
        #print(hotEId)
        
        findArticlesQuery = 'SELECT news_doc FROM news_data WHERE enter_id = %s'
        cur.execute(findArticlesQuery,coldEId)
        
        findArticles = cur.fetchall()
        
        for i in range(len(findArticles)):
            articles.append(findArticles[i][0])
        
        


    
        #sents = DoublespaceLineCorpus(corpus_path, iter_sent=True)

        noun_extractor = LRNounExtractor_v2(verbose=True)
        nouns = noun_extractor.train_extract(articles)
        return nouns

In [92]:
test = searchIssue()

In [79]:
test.hotEnter()

['아모레퍼시픽',
 '화학',
 53.85,
 53.85,
 [('거래', 213), ('자문', 111), ('올해', 93), ('참여', 89), ('회장', 86)]]

In [80]:
test.coldEnter()

['GS 건설',
 '건설',
 -14.6177,
 -14.6177,
 [('건설', 2926), ('PF', 1100), ('부동산', 700), ('분양', 680), ('워크아웃', 592)]]

In [94]:
k = test.test()

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 12980 from 164 sents. mem=0.559 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=67798, mem=0.562 Gb
[Noun Extractor] batch prediction was completed for 4365 words
[Noun Extractor] checked compounds. discovered 535 compounds
[Noun Extractor] postprocessing detaching_features : 2138 -> 2120
[Noun Extractor] postprocessing ignore_features : 2120 -> 2096
[Noun Extractor] postprocessing ignore_NJ : 2096 -> 2095
[Noun Extractor] 2095 nouns (535 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.571 Gb                    
[Noun Extractor] 64.82 % eojeols are covered


In [105]:
type(k['태영건설'])

soynlp.noun._noun_ver2.NounScore

In [117]:
t=sorted(k.values(),reverse=True)

In [126]:
print(sorted(k.items(), key=lambda x:x[1][0],reverse=True))

[('태영건설', NounScore(frequency=862, score=1.0)), ('PF', NounScore(frequency=766, score=1.0)), ('등', NounScore(frequency=731, score=1.0)), ('것', NounScore(frequency=536, score=0.9950372208436724)), ('부동산', NounScore(frequency=473, score=1.0)), ('워크아웃', NounScore(frequency=471, score=1.0)), ('수', NounScore(frequency=431, score=0.6666666666666666)), ('건설사', NounScore(frequency=389, score=1.0)), ('있다.', NounScore(frequency=364, score=1.0)), ('신청', NounScore(frequency=308, score=1.0)), ('신용등급', NounScore(frequency=287, score=1.0)), ('시장', NounScore(frequency=283, score=1.0)), ('가능성', NounScore(frequency=265, score=1.0)), ('분양', NounScore(frequency=250, score=1.0)), ('내년', NounScore(frequency=246, score=1.0)), ('중', NounScore(frequency=228, score=0.9240506329113924)), ('아파트', NounScore(frequency=227, score=0.896551724137931)), ('올해', NounScore(frequency=218, score=1.0)), ('대출', NounScore(frequency=212, score=1.0)), ('규모', NounScore(frequency=208, score=1.0)), ('문제', NounScore(frequency=204, s